In [20]:
# open and standardize image files, try stuff
import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
from sklearn.cluster import KMeans
from scipy import ndimage

dirs = ['plicatum_36','sargentii_17']

for d in dirs:
    files = []
    for file in glob.glob(d+'/*.ti*'):
        files.append(file)
    
    # open first image to get the image dimensions
    im = np.array(Image.open(files[0]))
    
    # define image_stack array
    image_stack = np.zeros([len(files),np.shape(im)[0],np.shape(im)[1]])
    # read and standardize all images
    for i in range(len(files)):
        im = np.array(Image.open(files[i]))        
        
        mean = np.mean(im)
        std = np.std(im)
        image_stack[i,:,:] = (im - mean)/std
        
        # for every nth images, explore
        if i%100 == 0:
            indx = files[i].index('.t')
            
            # histogram
            #hist, bin_edges = np.histogram(im,bins=1000,range=(0,50000))
            # to avoid log(0) later
            #hist[hist < 1] == 0.1
            #bins = (bin_edges[1:]+bin_edges[:-1])/2e0            
            
            #plt.close()
            #width = (bin_edges[1] - bin_edges[0])
            #center = (bin_edges[:-1] + bin_edges[1:]) / 2
            #plt.xlabel('pixel intensity')
            #plt.ylabel('log(count)')
            #plt.xlim([0,50000])
            #plt.ylim([0,5])
            #plt.bar(bin_edges[:-1], np.log10(hist), width)
            #plt.tight_layout(w_pad=0, h_pad=0)
            #plt.savefig('imgs/histo_'+d+'_'+files[i][indx-4:indx]+'.png')
            #plt.close()
            
            # LBP
            #lbp = local_binary_pattern(im, 8, 1)
            #plt.imshow(lbp)
            #plt.savefig('imgs/lbp_'+d+'_'+files[i][indx-4:indx]+'.png')
            #plt.close()            

            # k means clustering
            clustered_img = KMeans(2).fit_predict(im.reshape([np.prod(np.shape(im)),1]))            
            
            clustered_img[clustered_img == clustered_img[0]] = 2e0
            clustered_img[clustered_img != clustered_img[0]] = 3e0
            
            clustered_img = clustered_img.reshape(np.shape(im))
            plt.imshow(clustered_img)
            plt.savefig('imgs/kmeans_'+d+'_'+files[i][indx-4:indx]+'.png')
            plt.close()
            
            # label continouos regions
            labeled_img, num_regions = ndimage.label(clustered_img)
            
            plt.imshow(labeled_img)
            plt.savefig('imgs/labeled_'+d+'_'+files[i][indx-4:indx]+'.png')
            plt.close()
            
            # generate the size histogram of different regions
            histo = np.zeros(num_regions)
            for j in range(num_regions):
                histo[j] = np.sum(labeled_img == j)
            
            histo[histo == 0] = 0.1
            histo = np.sort(histo)[::-1]
            
            plt.xlabel('region label')
            plt.ylabel('region size')
            plt.xlim([0,20])
            plt.ylim([0,6])
            plt.bar(range(num_regions), np.log10(histo), np.zeros(num_regions)+1e0)
            plt.tight_layout(w_pad=0, h_pad=0)
            plt.savefig('imgs/area_histo_'+d+'_'+files[i][indx-4:indx]+'.png')
            plt.close()     


In [ ]:
# try marching cubes to extract the surface of the bud
import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from skimage import measure

dirs = ['plicatum_36','sargentii_17']

for d in dirs:
    files = []
    for file in glob.glob(d+'/*.ti*'):
        files.append(file)
    
    # open first image to get the image dimensions
    im = np.array(Image.open(files[0]))
    
    # define image_stack array
    image_stack = np.zeros([len(files),np.shape(im)[0],np.shape(im)[1]])
    # read images and cluster the pixels
    for i in range(len(files)):
        if i%20 == 0:
            print d,i
        
        # load image
        im = np.array(Image.open(files[i]))
        # do clustering
        clustered_img = KMeans(2).fit_predict(im.reshape([np.prod(np.shape(im)),1]))
        # make sure the background is 0, the bud is 1, clustering randomizes which one is which
        # assuming here that the corner pixel is always background
        if clustered_img[0] != 0:
            clustered_img ^= 1
        # reshape array
        clustered_img = clustered_img.reshape(np.shape(im))
        # save stack
        image_stack[i,:,:] = clustered_img

    # march the cubes
    verts, faces = measure.marching_cubes(image_stack, 0.5)
    
    # make the plot
    mesh = Poly3DCollection(verts[faces])
    ax.add_collection3d(mesh)
    ax.set_xlabel("x-axis")
    ax.set_ylabel("y-axis")
    ax.set_zlabel("z-axis")
    plt.savefig('imgs/marching_cubes_'+d+'.png')
    